In [1]:
import numpy as np
import matplotlib.pyplot as plt
from CRp_gamma import Phi_pp_gamma, velocity, integrate
# Takes CRp spectrum in MeV-1 cm-3 and returns gamma spectrum in erg cm-3 s-1
from CRe_gamma_radio import Phi_e_syn, Phi_e_rel_brem, Phi_e_IC
# Takes CRe spectrum in MeV-1 cm-3 and returns gamma spectrum in erg cm-3 s-1
from FeKalpha_CR import Phi_Fe_64_p, Phi_Fe_64_e
# Takes CR spectrum in MeV-1 cm-3 and returns flux in photons cm-2 s-1

## Parameters

General constants

In [4]:
pc = 3e18 #cm
kpc = 1e3 * pc 

MeV_to_erg = 1.602e-6 
m_p = 9.382e2 #MeV
m_e = 5.11e-1  #MeV
c = 2.998e10 #cm/s
hPlanck = 4.136e-21 #MeV/Hz
M_sol = 1.988e30 #kg
m_H = 1.674e-27 #kg
m_avg = 1.4 * m_H
sigma_sb = 3.539e1 #MeV cm-2 s-1 K-4

E_list = np.logspace(-13, 8, 100) #MeV

Cloud and shell parameters

In [5]:
ds = 2.2 * kpc

M_cl = 0.4 * M_sol
R_cl = 5.5e18 #cm (Miceli et al. 2016)
V_cl = (4 / 3) * np.pi * (R_cl ** 3)
n_cl = 0.5 #cm-3 (Miceli et al. 2016)
B_cl = 1e-2 #mG (few microG hence a maximum of 10 microG)

M_sh = 0.5 * M_sol
R_sh = 7.7 * pc #(Cristofari et al. 2019)
xf = 0.25 # volume-filling factor for SW limb
V_sh = (xf / 3) * np.pi * (R_sh ** 3) #cm3 (Cristofari et al. 2019)
n_sh = 0.035 #cm-3 (Giuffrida et al. 2007)
B_sh = 9e-2 #mG

Photon-field parameters

In [6]:
# T is K, w in MeV cm-3
T_CMB, w_CMB = 2.72, 0.261e-6
k_CMB = w_CMB * c / (sigma_sb * (T_CMB ** 4))
T_NIR, w_NIR = 30.0, 0.5e-6
k_NIR = w_NIR * c / (sigma_sb * (T_NIR ** 4))
T_FIR, w_FIR = 3000.0, 1.0e-6
k_FIR = w_FIR * c / (sigma_sb * (T_FIR ** 4))

## CR spectra

In [7]:
def Jp(p, delta, A, p_c):
    '''Gives no. of particle in MeV-3 cm-3'''
    return A * np.power(p, -delta) * np.exp(- p / p_c)


def J_CRp(T_p, del_p, A_p, p_pc):
    '''Gives no. of protons in MeV-1 cm-3 '''
    p_p = np.sqrt((T_p ** 2) + (2 * m_p * T_p))
    dp_on_dT = (T_p + m_p) / p_p
    return 4 * np.pi * (p_p ** 2) * Jp(p_p, del_p, A_p, p_pc) * dp_on_dT


def J_CRe(T_e, del_e, A_e, p_ec):
    '''Gives no. of electrons in MeV-1 cm-3 '''
    p_e = np.sqrt((T_e ** 2) + (2 * m_e * T_e))
    dp_on_dT = (T_e + m_e) / p_e
    return 4 * np.pi * (p_e ** 2) * Jp(p_e, del_e, A_e, p_ec) * dp_on_dT

## Load multi-wavelength data

Import data

In [ ]:
radio_data = np.genfromtxt(open("SN1006_data_radio.txt", "r"), delimiter=',') # in Hz and Jy Hz
